In [ ]:
import gc

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [ ]:
pd.set_option('mode.chained_assignment', None)

In [ ]:
NEW_NOTEBOOKS_PATH = "../../data/codeblocks_2021-04-01_concatenated.csv"
new_notebooks = pd.read_csv(NEW_NOTEBOOKS_PATH)#, nrows=25)
new_notebooks.rename({'data_sources':'ref'}, axis=1, inplace=True)
print('Parsed: {} chunks of {} notebooks from {} competitions'.format(new_notebooks.shape[0], new_notebooks['kaggle_link'].nunique(), new_notebooks['ref'].nunique()))
def clean_comp(string:str) -> str:
    string = string.strip('[').strip(']').replace("'", "")
    return string
new_notebooks['ref'] = new_notebooks['ref'].apply(clean_comp)
new_notebooks['ref'] = new_notebooks['ref'].apply(lambda x: x.split(',')[0].strip(' ').strip("'"))

In [ ]:
new_notebooks.drop_duplicates(inplace=True, subset=['code_block_id', 'kaggle_id'])
print(new_notebooks.shape, new_notebooks.kaggle_link.nunique())
new_notebooks.dropna(axis=0, subset=['code_block'], inplace=True)
print(new_notebooks.shape, new_notebooks.kaggle_link.nunique())
new_notebooks.dropna(axis=0, subset=['kaggle_score'], inplace=True) 
print(new_notebooks.shape, new_notebooks.kaggle_link.nunique())
gc.collect()

In [ ]:
new_notebooks['code_block'] = new_notebooks['code_block'].str.replace('`', '')

In [ ]:
COMPETITIONS_PATH = "../../data/competitions_info.csv"
competitions = pd.read_csv(COMPETITIONS_PATH)
print(competitions.shape)
competitions.drop_duplicates(inplace=True)
competitions['ref'] = competitions['ref'].apply(clean_comp)
print(competitions.shape)

In [ ]:
new_notebooks = new_notebooks.merge(competitions, on=['ref'], how='inner')
print('# Chunks: {}\n# Notebooks: {}\n# Competitions: {}'.format(new_notebooks.shape[0], new_notebooks.kaggle_link.nunique(), new_notebooks.ref.nunique()))

In [ ]:
# new_notebooks.drop('Unnamed: 0', axis=1, inplace=True)
# new_notebooks.to_csv('../../data/codeblocks_2021-04-01_concatenated_cleaned.csv', index=False)

In [ ]:
# new_notebooks['exists_in_competitions'] = new_notebooks.apply(lambda x: x['ref'] in competitions['ref'].unique(), axis=1)
# print('# Chunks: {}\n# Notebooks: {}\n# Competitions: {}'.format(new_notebooks[new_notebooks['exists_in_competitions'] == True].shape[0], new_notebooks[new_notebooks['exists_in_competitions'] == True]['kaggle_link'].nunique(), new_notebooks[new_notebooks['exists_in_competitions'] == True]['ref'].nunique()))

In [ ]:
competitions['has_notebooks'] = competitions.apply(lambda x: x['ref'] in new_notebooks['ref'].tolist(), axis=1)
competitions_cleaned = competitions[competitions['has_notebooks']]
competitions_cleaned.shape
# competitions_cleaned.to_csv('../data/competitions_info_cleaned.csv', index=False)

In [ ]:
COMPETITIONS_FILLED_PATH = "../../data/competitions_info_cleaned.csv"
competitions_filled = pd.read_csv(COMPETITIONS_FILLED_PATH)
competitions_filled.shape

In [ ]:
competitions_cleaned_with_filled = competitions_cleaned[['ref', 'comp_name', 'comp_type']].merge(competitions_filled[['ref', 'Description',
                                        'Metric', 'DataType', 'Subject', 'ProblemType']]
                                    , on=['ref'], how='left')
competitions_cleaned_with_filled.isna().sum()

In [ ]:
# competitions_filled.to_csv('../../data/competitions_info_filled.csv', index=False)
# competitions_filled.to_csv('../../data_mini/competitions_info_filled.csv', index=False)

In [ ]:
# competitions_cleaned.to_csv('../../data/competitions_info_cleaned.csv', index=False)
# competitions_cleaned.to_csv('../../data_mini/competitions_info_cleaned.csv', index=False)

In [ ]:
# competitions_cleaned_with_filled.to_csv('../../data/competitions_info_cleaned_filled.csv', index=False)
# competitions_cleaned_with_filled.to_csv('../../data_mini/competitions_info_cleaned_filled.csv', index=False)